# 🛡️ Honeypot Attack Analysis - Interactive Dashboard

**Phase 7: Advanced Analytics with Jupyter**

This notebook provides interactive visualizations and analysis of honeypot attack data, including:

- 📊 Top attacking IP addresses
- ⏰ Attack patterns over time
- 🔑 Most attempted credentials
- 🌍 Geographic distribution of attacks
- 🚨 Threat detection and alerting

---

## 1️⃣ Import Required Libraries

Installing and importing all necessary libraries for data analysis and visualization.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import json
import sqlite3
from datetime import datetime, timedelta
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

## 2️⃣ Load and Parse Data

Loading attack data from the SQLite database created by the analysis pipeline.

In [ ]:
# Configuration
DB_PATH = 'out/results.db'

# Connect to SQLite database
conn = sqlite3.connect(DB_PATH)

# Load main events data
df_events = pd.read_sql_query("""
    SELECT 
        timestamp,
        received_at,
        ip,
        username,
        password_hash,
        path,
        method,
        user_agent,
        geo_country,
        geo_city,
        geo_lat,
        geo_lon,
        asn_number,
        asn_org,
        rdns,
        tags,
        confidence_geo,
        confidence_asn
    FROM events
    ORDER BY timestamp
""", conn)

# Convert timestamp columns
df_events['timestamp'] = pd.to_datetime(df_events['timestamp'])
df_events['received_at'] = pd.to_datetime(df_events['received_at'])

# Load summary statistics
summary = pd.read_sql_query("SELECT * FROM summary", conn).iloc[0]

print("✅ Data loaded successfully!")
print(f"\n📊 Dataset Overview:")
print(f"   Total Events: {len(df_events):,}")
print(f"   Date Range: {df_events['timestamp'].min()} to {df_events['timestamp'].max()}")
print(f"   Unique IPs: {df_events['ip'].nunique():,}")
print(f"   Countries: {df_events['geo_country'].nunique():,}")
print(f"\n📈 First few events:")
df_events.head()

## 3️⃣ Top Attacking IPs

Identifying and visualizing the most aggressive attacking IP addresses.

In [ ]:
# Aggregate attacks by IP
top_ips = df_events.groupby(['ip', 'geo_country', 'asn_org']).agg({
    'timestamp': 'count',
    'username': lambda x: x.nunique(),
    'path': lambda x: x.nunique()
}).rename(columns={
    'timestamp': 'attempts',
    'username': 'unique_usernames',
    'path': 'unique_paths'
}).reset_index().sort_values('attempts', ascending=False).head(20)

print("🎯 Top 20 Attacking IPs:\n")
print(top_ips.to_string(index=False))

# Interactive bar chart
fig = px.bar(
    top_ips.head(15),
    x='attempts',
    y='ip',
    color='geo_country',
    orientation='h',
    title='🔴 Top 15 Attacking IP Addresses',
    labels={'attempts': 'Number of Attacks', 'ip': 'IP Address', 'geo_country': 'Country'},
    text='attempts',
    hover_data=['asn_org', 'unique_usernames', 'unique_paths']
)

fig.update_layout(
    height=600,
    yaxis={'categoryorder': 'total ascending'},
    showlegend=True,
    template='plotly_dark'
)

fig.update_traces(textposition='outside')
fig.show()

## 4️⃣ Attack Patterns Over Time

Analyzing when attacks occur - daily trends, hourly patterns, and peak activity times.

In [ ]:
# Extract time components
df_events['date'] = df_events['timestamp'].dt.date
df_events['hour'] = df_events['timestamp'].dt.hour
df_events['day_of_week'] = df_events['timestamp'].dt.day_name()

# Daily attack counts
daily_attacks = df_events.groupby('date').size().reset_index(name='attacks')
daily_attacks['date'] = pd.to_datetime(daily_attacks['date'])

# Hourly distribution
hourly_attacks = df_events.groupby('hour').size().reset_index(name='attacks')

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('📅 Daily Attack Trends', '🕐 Hourly Attack Distribution'),
    vertical_spacing=0.15,
    row_heights=[0.5, 0.5]
)

# Daily trend line chart
fig.add_trace(
    go.Scatter(
        x=daily_attacks['date'],
        y=daily_attacks['attacks'],
        mode='lines+markers',
        name='Daily Attacks',
        line=dict(color='#ef4444', width=3),
        marker=dict(size=8),
        fill='tozeroy',
        fillcolor='rgba(239, 68, 68, 0.2)'
    ),
    row=1, col=1
)

# Hourly bar chart
fig.add_trace(
    go.Bar(
        x=hourly_attacks['hour'],
        y=hourly_attacks['attacks'],
        name='Hourly Attacks',
        marker_color='#3b82f6',
        text=hourly_attacks['attacks'],
        textposition='outside'
    ),
    row=2, col=1
)

fig.update_layout(
    height=800,
    showlegend=False,
    template='plotly_dark'
)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Hour of Day", row=2, col=1)
fig.update_yaxes(title_text="Number of Attacks", row=1, col=1)
fig.update_yaxes(title_text="Number of Attacks", row=2, col=1)

fig.show()

# Print statistics
peak_day = daily_attacks.loc[daily_attacks['attacks'].idxmax()]
peak_hour = hourly_attacks.loc[hourly_attacks['attacks'].idxmax()]

print(f"\n📊 Time Pattern Statistics:")
print(f"   Peak Day: {peak_day['date']} with {peak_day['attacks']} attacks")
print(f"   Peak Hour: {peak_hour['hour']}:00 with {peak_hour['attacks']} attacks")
print(f"   Average per Day: {daily_attacks['attacks'].mean():.1f}")
print(f"   Average per Hour: {hourly_attacks['attacks'].mean():.1f}")

## 5️⃣ Credential Harvesting Analysis

Most commonly attempted usernames and passwords (hashed for security).

In [ ]:
# Top usernames
top_usernames = df_events['username'].value_counts().head(20).reset_index()
top_usernames.columns = ['username', 'count']

# Top password hashes (for security, we show hashes not actual passwords)
top_passwords = df_events['password_hash'].value_counts().head(20).reset_index()
top_passwords.columns = ['password_hash', 'count']

# Create side-by-side charts
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('👤 Top 20 Attempted Usernames', '🔑 Top 20 Password Patterns (Hashed)'),
    horizontal_spacing=0.1
)

# Usernames
fig.add_trace(
    go.Bar(
        y=top_usernames['username'],
        x=top_usernames['count'],
        orientation='h',
        marker_color='#10b981',
        text=top_usernames['count'],
        textposition='outside',
        name='Usernames'
    ),
    row=1, col=1
)

# Passwords
fig.add_trace(
    go.Bar(
        y=top_passwords['password_hash'].str[:12] + '...',  # Truncate for display
        x=top_passwords['count'],
        orientation='h',
        marker_color='#f59e0b',
        text=top_passwords['count'],
        textposition='outside',
        name='Passwords'
    ),
    row=1, col=2
)

fig.update_layout(
    height=600,
    showlegend=False,
    template='plotly_dark'
)

fig.update_yaxes(categoryorder='total ascending', row=1, col=1)
fig.update_yaxes(categoryorder='total ascending', row=1, col=2)

fig.show()

print(f"\n🔐 Credential Statistics:")
print(f"   Unique Usernames: {df_events['username'].nunique():,}")
print(f"   Unique Passwords: {df_events['password_hash'].nunique():,}")
print(f"   Most Common Username: '{top_usernames.iloc[0]['username']}' ({top_usernames.iloc[0]['count']} attempts)")
print(f"\n📋 Top 10 Usernames:")
for i, row in top_usernames.head(10).iterrows():
    print(f"   {i+1}. {row['username']:<20} - {row['count']:>4} attempts")

## 6️⃣ Geographic Distribution

Visualizing attack origins worldwide - where are the threats coming from?

In [ ]:
# Country distribution
country_attacks = df_events.groupby('geo_country').agg({
    'ip': 'count',
    'geo_lat': 'first',
    'geo_lon': 'first'
}).rename(columns={'ip': 'attacks'}).reset_index()
country_attacks = country_attacks.sort_values('attacks', ascending=False)

# World map with attack intensity
fig = px.choropleth(
    country_attacks,
    locations='geo_country',
    locationmode='country names',
    color='attacks',
    hover_name='geo_country',
    hover_data={'attacks': True, 'geo_country': False},
    color_continuous_scale='Reds',
    title='🌍 Global Attack Distribution Map'
)

fig.update_layout(
    height=500,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    template='plotly_dark'
)

fig.show()

# Pie chart for top countries
top_countries = country_attacks.head(10)
other_attacks = country_attacks.iloc[10:]['attacks'].sum()

if other_attacks > 0:
    top_countries = pd.concat([
        top_countries,
        pd.DataFrame([{'geo_country': 'Others', 'attacks': other_attacks}])
    ])

fig2 = px.pie(
    top_countries,
    values='attacks',
    names='geo_country',
    title='🎯 Top Countries by Attack Volume',
    color_discrete_sequence=px.colors.sequential.Reds_r
)

fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.update_layout(height=500, template='plotly_dark')
fig2.show()

print(f"\n🌍 Geographic Statistics:")
print(f"   Countries Involved: {country_attacks['geo_country'].nunique():,}")
print(f"   Top 10 Countries:\n")
for i, row in country_attacks.head(10).iterrows():
    pct = (row['attacks'] / len(df_events)) * 100
    print(f"   {row['geo_country']:<30} {row['attacks']:>6} attacks ({pct:>5.1f}%)")

## 7️⃣ Threat Detection - High Frequency Attacks

**Phase 8 Integration:** Identifying IPs with suspicious activity (≥10 attempts in 5-minute windows).

In [ ]:
# Detect high-frequency attacks (10+ attempts in 5-minute window)
WINDOW_MINUTES = 5
THRESHOLD = 10

# Sort by IP and timestamp
df_sorted = df_events.sort_values(['ip', 'timestamp']).copy()

# Calculate time difference between consecutive events from same IP
df_sorted['time_diff'] = df_sorted.groupby('ip')['timestamp'].diff()

# Detect bursts: find sequences where time_diff < 5 minutes
threat_ips = []

for ip in df_sorted['ip'].unique():
    ip_events = df_sorted[df_sorted['ip'] == ip].copy()
    
    if len(ip_events) < THRESHOLD:
        continue
    
    # Sliding window detection
    for i in range(len(ip_events) - THRESHOLD + 1):
        window_events = ip_events.iloc[i:i+THRESHOLD]
        time_span = window_events['timestamp'].max() - window_events['timestamp'].min()
        
        if time_span <= timedelta(minutes=WINDOW_MINUTES):
            threat_info = {
                'ip': ip,
                'country': ip_events.iloc[0]['geo_country'],
                'asn_org': ip_events.iloc[0]['asn_org'],
                'attempts': len(window_events),
                'window_start': window_events['timestamp'].min(),
                'window_end': window_events['timestamp'].max(),
                'time_span_seconds': time_span.total_seconds(),
                'top_usernames': window_events['username'].value_counts().head(3).to_dict(),
                'tags': window_events['tags'].mode()[0] if len(window_events['tags'].mode()) > 0 else 'unknown'
            }
            threat_ips.append(threat_info)
            break  # Only count once per IP

# Create threat DataFrame
df_threats = pd.DataFrame(threat_ips)

if len(df_threats) > 0:
    print(f"🚨 HIGH-RISK THREATS DETECTED: {len(df_threats)}\n")
    print(f"{'IP Address':<20} {'Country':<15} {'Attempts':<10} {'Time Span':<15} {'Top Username'}")
    print("=" * 85)
    
    for _, threat in df_threats.iterrows():
        top_user = list(threat['top_usernames'].keys())[0] if threat['top_usernames'] else 'N/A'
        print(f"{threat['ip']:<20} {threat['country']:<15} {threat['attempts']:<10} "
              f"{threat['time_span_seconds']:.0f}s{'':<11} {top_user}")
    
    # Visualize threat levels
    fig = px.bar(
        df_threats.sort_values('attempts', ascending=False).head(15),
        x='ip',
        y='attempts',
        color='time_span_seconds',
        title=f'🚨 High-Risk IPs (≥{THRESHOLD} attempts in {WINDOW_MINUTES} minutes)',
        labels={'attempts': 'Attack Attempts', 'ip': 'IP Address', 'time_span_seconds': 'Time Span (seconds)'},
        text='attempts',
        color_continuous_scale='Reds'
    )
    
    fig.update_layout(
        height=500,
        xaxis_tickangle=-45,
        template='plotly_dark'
    )
    
    fig.show()
    
else:
    print(f"✅ No high-frequency attacks detected (threshold: {THRESHOLD} attempts in {WINDOW_MINUTES} minutes)")

## 8️⃣ Alert Payload Generation

**Phase 8 Implementation:** Generate Slack/Email alert payloads for detected threats.

In [ ]:
def generate_slack_alert(threat):
    """
    Generate Slack webhook payload for a detected threat.
    
    Args:
        threat: Dictionary containing threat information
        
    Returns:
        JSON-formatted Slack payload
    """
    # Format top usernames
    top_users = ', '.join([f"{k} ({v})" for k, v in list(threat['top_usernames'].items())[:3]])
    
    # Create Slack message blocks
    payload = {
        "text": f"🚨 Honeypot Brute-Force Alert from {threat['ip']}",
        "blocks": [
            {
                "type": "header",
                "text": {
                    "type": "plain_text",
                    "text": "🚨 Honeypot Alert - High Frequency Attack Detected",
                    "emoji": True
                }
            },
            {
                "type": "section",
                "fields": [
                    {
                        "type": "mrkdwn",
                        "text": f"*IP Address:*\n`{threat['ip']}`"
                    },
                    {
                        "type": "mrkdwn",
                        "text": f"*Attempts:*\n{threat['attempts']} in {threat['time_span_seconds']:.0f}s"
                    },
                    {
                        "type": "mrkdwn",
                        "text": f"*Country:*\n{threat['country']}"
                    },
                    {
                        "type": "mrkdwn",
                        "text": f"*Organization:*\n{threat['asn_org'][:30]}..."
                    }
                ]
            },
            {
                "type": "section",
                "fields": [
                    {
                        "type": "mrkdwn",
                        "text": f"*Top Usernames:*\n{top_users}"
                    },
                    {
                        "type": "mrkdwn",
                        "text": f"*Attack Tags:*\n{threat['tags']}"
                    }
                ]
            },
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"*Time Window:*\n{threat['window_start']} → {threat['window_end']}"
                }
            },
            {
                "type": "divider"
            },
            {
                "type": "context",
                "elements": [
                    {
                        "type": "mrkdwn",
                        "text": f"⚠️ Automatic alert generated at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
                    }
                ]
            }
        ]
    }
    
    return payload


# Generate alerts for all detected threats
if len(df_threats) > 0:
    print("📧 GENERATED ALERT PAYLOADS:\n")
    print("=" * 80)
    
    for idx, threat in df_threats.iterrows():
        alert_payload = generate_slack_alert(threat)
        
        print(f"\n🎯 Alert #{idx + 1} - IP: {threat['ip']}")
        print("-" * 80)
        print(json.dumps(alert_payload, indent=2))
        print("-" * 80)
        
        # Save to file
        alert_filename = f"out/alert_{threat['ip'].replace('.', '_')}_{threat['window_start'].strftime('%Y%m%d_%H%M%S')}.json"
        with open(alert_filename, 'w') as f:
            json.dump(alert_payload, f, indent=2)
        print(f"💾 Saved to: {alert_filename}\n")
    
    print(f"\n✅ Generated {len(df_threats)} alert payloads")
    print(f"\n📝 To send to Slack:")
    print(f"   curl -X POST -H 'Content-Type: application/json' \\")
    print(f"        -d @out/alert_<filename>.json \\")
    print(f"        https://hooks.slack.com/services/YOUR/WEBHOOK/URL")
    
else:
    print("✅ No threats detected - no alerts generated")

## 9️⃣ Summary Dashboard

Complete overview of all honeypot metrics and statistics.

In [ ]:
print("=" * 80)
print("🛡️  HONEYPOT ANALYSIS SUMMARY")
print("=" * 80)

print(f"\n📊 OVERALL STATISTICS:")
print(f"   Total Attack Events:      {len(df_events):,}")
print(f"   Date Range:               {df_events['timestamp'].min()} to {df_events['timestamp'].max()}")
print(f"   Days Observed:            {(df_events['timestamp'].max() - df_events['timestamp'].min()).days + 1}")
print(f"   Average per Day:          {len(df_events) / ((df_events['timestamp'].max() - df_events['timestamp'].min()).days + 1):.1f}")

print(f"\n🌐 SOURCE ANALYSIS:")
print(f"   Unique IP Addresses:      {df_events['ip'].nunique():,}")
print(f"   Countries Involved:       {df_events['geo_country'].nunique()}")
print(f"   ASN Organizations:        {df_events['asn_org'].nunique()}")
print(f"   Average Attempts per IP:  {len(df_events) / df_events['ip'].nunique():.1f}")

print(f"\n🔑 CREDENTIAL ANALYSIS:")
print(f"   Unique Usernames Tried:   {df_events['username'].nunique():,}")
print(f"   Unique Passwords Tried:   {df_events['password_hash'].nunique():,}")
print(f"   Most Common Username:     '{top_usernames.iloc[0]['username']}' ({top_usernames.iloc[0]['count']} times)")

print(f"\n⏰ TEMPORAL PATTERNS:")
print(f"   Peak Day:                 {peak_day['date']} ({peak_day['attacks']} attacks)")
print(f"   Peak Hour:                {peak_hour['hour']}:00 ({peak_hour['attacks']} attacks)")
print(f"   Most Active Day of Week:  {df_events['day_of_week'].mode()[0]}")

print(f"\n🎯 ATTACK PATTERNS:")
print(f"   Most Targeted Path:       {df_events['path'].mode()[0]}")
print(f"   Most Common Method:       {df_events['method'].mode()[0]}")
print(f"   Attack Tags Found:        {df_events['tags'].nunique()}")

if len(df_threats) > 0:
    print(f"\n🚨 THREAT DETECTION:")
    print(f"   High-Risk IPs Detected:   {len(df_threats)}")
    print(f"   Alert Threshold:          {THRESHOLD} attempts in {WINDOW_MINUTES} minutes")
    print(f"   Alerts Generated:         {len(df_threats)}")
else:
    print(f"\n✅ THREAT DETECTION:")
    print(f"   No high-frequency attacks detected")

print(f"\n🌍 TOP 5 ATTACKING COUNTRIES:")
for i, row in country_attacks.head(5).iterrows():
    pct = (row['attacks'] / len(df_events)) * 100
    bar = '█' * int(pct / 2)
    print(f"   {row['geo_country']:<20} {bar:<50} {row['attacks']:>6} ({pct:>5.1f}%)")

print(f"\n📈 DATA QUALITY:")
print(f"   Events with GeoIP:        {df_events['geo_country'].notna().sum()} ({df_events['geo_country'].notna().sum()/len(df_events)*100:.1f}%)")
print(f"   Events with ASN:          {df_events['asn_org'].notna().sum()} ({df_events['asn_org'].notna().sum()/len(df_events)*100:.1f}%)")
print(f"   Events with Tags:         {df_events['tags'].notna().sum()} ({df_events['tags'].notna().sum()/len(df_events)*100:.1f}%)")

print("\n" + "=" * 80)
print("✅ Analysis Complete!")
print("=" * 80)